In [16]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import torch
from PIL import Image
import cv2
from torchvision import transforms

# Función para cargar modelos PyTorch
def cargar_modelo_pytorch(ruta):
    modelo = torch.load(ruta)
    modelo.eval()  # Pone el modelo en modo evaluación
    return modelo

# Definir el preprocesamiento para los modelos .pth
transform_pth = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Función para preprocesar imágenes para modelos .h5
def preprocesar_imagen_h5(img):
    img_redimensionada = redimensionar_imagen(img, (256, 256))
    img_normalizada = img_redimensionada / 255.0
    return np.expand_dims(img_normalizada, axis=0)

# Función para preprocesar imágenes para modelos .pth
def preprocesar_imagen_pth(img):
    img_tensor = transform_pth(img).unsqueeze(0)
    return img_tensor

# Función para redimensionar la imagen
def redimensionar_imagen(img, nuevo_tamanio):
    if img.dtype != np.uint8:
        img = (img * 255).astype(np.uint8)
    img_pil = Image.fromarray(img)
    img_redimensionada = img_pil.resize(nuevo_tamanio, Image.Resampling.LANCZOS)
    return np.array(img_redimensionada)

# Cargar la imagen desde la carpeta
def cargar_imagen_de_carpeta(ruta_carpeta, nombre_archivo):
    ruta_imagen = os.path.join(ruta_carpeta, nombre_archivo)
    img = cv2.imread(ruta_imagen)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Codificaciones para cada modelo
codificaciones = {
    'Propio': {0:'Basket', 1:'Campo_Futbol', 2:'Cancha_Multiple', 3: 'Parque', 4:'Tenis'},
    'PatternNET': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
    'Optimal31': {0: 'baseball_diamond', 1: 'basketball_court', 2: 'golf_course', 3: 'ground_track_field'},
    'WHU': {0: 'baseballdiamond', 1: 'footballField', 2: 'golfcourse', 3: 'Park', 4: 'Pond', 5: 'tenniscourt'},
    'Mixto': {0: 'baseball_diamond', 1: 'Basket', 2: 'Campo_Futbol', 3: 'Cancha_Multiple', 4: 'golf_course', 5: 'Parque', 6: 'Tenis'},
    'MLRS' : {'baseball_diamond': 0, 'basketball_court': 1, 'golf_course': 2, 'ground_track_field': 3, 'park': 4,'stadium': 5,'tennis_court': 6}
}

# Cargar modelos
modelos = {
    'Propio': load_model('C:/Users/nicol/Desktop/Modelos/Propio/Propio_Xception.h5'),
    'PatternNET': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/PatternNET/PatternNET_mobileNET.pth'),
    'Optimal31': load_model('C:/Users/nicol/Desktop/Modelos/Optimal31/Optimal31_VGG16.h5'),
    'WHU': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/WHU/WHU_mobileNET.pth'),
    'Mixto': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/Mixto/Mixto_mobileNET.pth'),
    'MLRSNet' : cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/MLSRNet/MLRSNet_mobileNET.pth')
}

ruta_carpeta = 'C:/Users/nicol/Desktop/Tests'
archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]

# Preparar DataFrame
df = pd.DataFrame(columns=['Imagen'] + list(modelos.keys()))
df['Imagen'] = archivos

# Realizar predicciones
for archivo in archivos:
    img = cargar_imagen_de_carpeta(ruta_carpeta, archivo)

    for nombre_modelo, modelo in modelos.items():
        if isinstance(modelo, tf.keras.Model):
            img_preprocesada = preprocesar_imagen_h5(img)
            pred = modelo.predict(img_preprocesada)
            pred_clase = np.argmax(pred, axis=1)[0]
        elif isinstance(modelo, torch.nn.Module):
            img_preprocesada = preprocesar_imagen_pth(img)
            with torch.no_grad():
                pred = modelo(img_preprocesada)
                pred_clase = pred.argmax(dim=1).item()
        
        # Convertir el índice predicho a la clase correspondiente usando la codificación
        clase_decodificada = codificaciones[nombre_modelo].get(pred_clase, 'Desconocido')
        df.loc[df['Imagen'] == archivo, nombre_modelo] = clase_decodificada

df


C:\Users\nicol\AppData\Local\Temp\ipykernel_11784\1218662245.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo = torch.load(ruta)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 729ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/ste

,Imagen,Propio,PatternNET,Optimal31,WHU,Mixto
0,10_Basket.jpg,Basket,basketball_court,basketball_court,baseballdiamond,Basket
1,11_Tenis.JPG,Tenis,tennis_court,basketball_court,tenniscourt,Tenis
2,12_Multiples_Basket_Tenis.JPG,Tenis,basketball_court,basketball_court,tenniscourt,Basket
3,13_Multiple_Basket_Tenis.jpg,Tenis,tennis_court,basketball_court,Park,Tenis
4,14_Tenis_Varias.jpg,Tenis,football_field,basketball_court,golfcourse,Tenis
5,15_Tenis_Varias.jpg,Tenis,tennis_court,basketball_court,tenniscourt,Tenis
6,16_Multiples.JPG,Tenis,tennis_court,basketball_court,tenniscourt,Tenis
7,17_Multiples.jpg,Campo_Futbol,basketball_court,ground_track_field,tenniscourt,Campo_Futbol
8,1_Baseball.tif,Tenis,baseball_field,baseball_diamond,baseballdiamond,baseball_diamond
9,2_Park.jpg,Cancha_Multiple,basketball_court,golf_course,Park,Parque


# Propio

In [17]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import torch
from PIL import Image
import cv2
from torchvision import transforms

# Función para cargar modelos PyTorch
def cargar_modelo_pytorch(ruta):
    modelo = torch.load(ruta)
    modelo.eval()  # Pone el modelo en modo evaluación
    return modelo

# Definir el preprocesamiento para los modelos .pth
transform_pth = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Función para preprocesar imágenes para modelos .h5
def preprocesar_imagen_h5(img):
    img_redimensionada = redimensionar_imagen(img, (256, 256))
    img_normalizada = img_redimensionada / 255.0
    return np.expand_dims(img_normalizada, axis=0)

# Función para preprocesar imágenes para modelos .pth
def preprocesar_imagen_pth(img):
    img_tensor = transform_pth(img).unsqueeze(0)
    return img_tensor

# Función para redimensionar la imagen
def redimensionar_imagen(img, nuevo_tamanio):
    if img.dtype != np.uint8:
        img = (img * 255).astype(np.uint8)
    img_pil = Image.fromarray(img)
    img_redimensionada = img_pil.resize(nuevo_tamanio, Image.Resampling.LANCZOS)
    return np.array(img_redimensionada)

# Cargar la imagen desde la carpeta
def cargar_imagen_de_carpeta(ruta_carpeta, nombre_archivo):
    ruta_imagen = os.path.join(ruta_carpeta, nombre_archivo)
    img = cv2.imread(ruta_imagen)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Codificaciones para cada modelo
codificaciones = {
    'Propio_1': {0:'Basket', 1:'Campo_Futbol', 2:'Cancha_Multiple', 3: 'Parque', 4:'Tenis'},
    'Propio_2': {0:'Basket', 1:'Campo_Futbol', 2:'Cancha_Multiple', 3: 'Parque', 4:'Tenis'},
    'Propio_3': {0:'Basket', 1:'Campo_Futbol', 2:'Cancha_Multiple', 3: 'Parque', 4:'Tenis'},
    'Propio_4': {0:'Basket', 1:'Campo_Futbol', 2:'Cancha_Multiple', 3: 'Parque', 4:'Tenis'},
    'Propio_5': {0:'Basket', 1:'Campo_Futbol', 2:'Cancha_Multiple', 3: 'Parque', 4:'Tenis'},
}

# Cargar modelos
modelos = {
    'Propio_1': load_model('C:/Users/nicol/Desktop/Modelos/Propio/Propio_Custom.h5'),
    'Propio_2': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/Propio/Propio_mobileNET.pth'),
    'Propio_3': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/Propio/Propio_ResNET18.pth'),
    'Propio_4': load_model('C:/Users/nicol/Desktop/Modelos/Propio/Propio_VGG16.h5'),
    'Propio_5': load_model('C:/Users/nicol/Desktop/Modelos/Propio/Propio_Xception.h5'),
}

ruta_carpeta = 'C:/Users/nicol/Desktop/Tests'
archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]

# Preparar DataFrame
df = pd.DataFrame(columns=['Imagen'] + list(modelos.keys()))
df['Imagen'] = archivos

# Realizar predicciones
for archivo in archivos:
    img = cargar_imagen_de_carpeta(ruta_carpeta, archivo)

    for nombre_modelo, modelo in modelos.items():
        if isinstance(modelo, tf.keras.Model):
            img_preprocesada = preprocesar_imagen_h5(img)
            pred = modelo.predict(img_preprocesada)
            pred_clase = np.argmax(pred, axis=1)[0]
        elif isinstance(modelo, torch.nn.Module):
            img_preprocesada = preprocesar_imagen_pth(img)
            with torch.no_grad():
                pred = modelo(img_preprocesada)
                pred_clase = pred.argmax(dim=1).item()
        
        # Convertir el índice predicho a la clase correspondiente usando la codificación
        clase_decodificada = codificaciones[nombre_modelo].get(pred_clase, 'Desconocido')
        df.loc[df['Imagen'] == archivo, nombre_modelo] = clase_decodificada

df


C:\Users\nicol\AppData\Local\Temp\ipykernel_11784\2107530662.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo = torch.load(ruta)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 709ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/

,Imagen,Propio_1,Propio_2,Propio_3,Propio_4,Propio_5
0,10_Basket.jpg,Tenis,Basket,Basket,Basket,Basket
1,11_Tenis.JPG,Tenis,Tenis,Tenis,Tenis,Tenis
2,12_Multiples_Basket_Tenis.JPG,Tenis,Basket,Basket,Cancha_Multiple,Tenis
3,13_Multiple_Basket_Tenis.jpg,Cancha_Multiple,Campo_Futbol,Campo_Futbol,Campo_Futbol,Tenis
4,14_Tenis_Varias.jpg,Tenis,Tenis,Tenis,Tenis,Tenis
5,15_Tenis_Varias.jpg,Basket,Tenis,Campo_Futbol,Tenis,Tenis
6,16_Multiples.JPG,Tenis,Tenis,Campo_Futbol,Tenis,Tenis
7,17_Multiples.jpg,Campo_Futbol,Campo_Futbol,Campo_Futbol,Campo_Futbol,Campo_Futbol
8,1_Baseball.tif,Parque,Campo_Futbol,Campo_Futbol,Tenis,Tenis
9,2_Park.jpg,Basket,Campo_Futbol,Parque,Cancha_Multiple,Cancha_Multiple


# PatterNET

In [19]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import torch
from PIL import Image
import cv2
from torchvision import transforms

# Función para cargar modelos PyTorch
def cargar_modelo_pytorch(ruta):
    modelo = torch.load(ruta)
    modelo.eval()  # Pone el modelo en modo evaluación
    return modelo

# Definir el preprocesamiento para los modelos .pth
transform_pth = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Función para preprocesar imágenes para modelos .h5
def preprocesar_imagen_h5(img):
    img_redimensionada = redimensionar_imagen(img, (256, 256))
    img_normalizada = img_redimensionada / 255.0
    return np.expand_dims(img_normalizada, axis=0)

# Función para preprocesar imágenes para modelos .pth
def preprocesar_imagen_pth(img):
    img_tensor = transform_pth(img).unsqueeze(0)
    return img_tensor

# Función para redimensionar la imagen
def redimensionar_imagen(img, nuevo_tamanio):
    if img.dtype != np.uint8:
        img = (img * 255).astype(np.uint8)
    img_pil = Image.fromarray(img)
    img_redimensionada = img_pil.resize(nuevo_tamanio, Image.Resampling.LANCZOS)
    return np.array(img_redimensionada)

# Cargar la imagen desde la carpeta
def cargar_imagen_de_carpeta(ruta_carpeta, nombre_archivo):
    ruta_imagen = os.path.join(ruta_carpeta, nombre_archivo)
    img = cv2.imread(ruta_imagen)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Codificaciones para cada modelo
codificaciones = {
    'Pattern_Custom': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
    'Pattern_Mobile': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
    'Pattern_Resnet': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
    'Pattern_VG16': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
    'Pattern_Xception': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
}

# Cargar modelos
modelos = {
    'Pattern_Custom': load_model('C:/Users/nicol/Desktop/Modelos/PatternNET/PatternNET_Custom.h5'),
    'Pattern_Mobile': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/PatternNET/PatternNET_mobileNET.pth'),
    'Pattern_Resnet': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/PatternNET/PatternNET_ResNET18.pth'),
    'Pattern_VG16': load_model('C:/Users/nicol/Desktop/Modelos/PatternNET/PatternNET_VGG16.h5'),
    'Pattern_Xception': load_model('C:/Users/nicol/Desktop/Modelos/PatternNET/PatternNET_Xception.h5'),
}

ruta_carpeta = 'C:/Users/nicol/Desktop/Tests'
archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]

# Preparar DataFrame
df = pd.DataFrame(columns=['Imagen'] + list(modelos.keys()))
df['Imagen'] = archivos

# Realizar predicciones
for archivo in archivos:
    img = cargar_imagen_de_carpeta(ruta_carpeta, archivo)

    for nombre_modelo, modelo in modelos.items():
        if isinstance(modelo, tf.keras.Model):
            img_preprocesada = preprocesar_imagen_h5(img)
            pred = modelo.predict(img_preprocesada)
            pred_clase = np.argmax(pred, axis=1)[0]
        elif isinstance(modelo, torch.nn.Module):
            img_preprocesada = preprocesar_imagen_pth(img)
            with torch.no_grad():
                pred = modelo(img_preprocesada)
                pred_clase = pred.argmax(dim=1).item()
        
        # Convertir el índice predicho a la clase correspondiente usando la codificación
        clase_decodificada = codificaciones[nombre_modelo].get(pred_clase, 'Desconocido')
        df.loc[df['Imagen'] == archivo, nombre_modelo] = clase_decodificada

df


C:\Users\nicol\AppData\Local\Temp\ipykernel_11784\917178343.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo = torch.load(ruta)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/

,Imagen,Pattern_Custom,Pattern_Mobile,Pattern_Resnet,Pattern_VG16,Pattern_Xception
0,10_Basket.jpg,basketball_court,basketball_court,basketball_court,basketball_court,basketball_court
1,11_Tenis.JPG,football_field,tennis_court,tennis_court,tennis_court,tennis_court
2,12_Multiples_Basket_Tenis.JPG,basketball_court,basketball_court,basketball_court,basketball_court,tennis_court
3,13_Multiple_Basket_Tenis.jpg,tennis_court,tennis_court,tennis_court,basketball_court,tennis_court
4,14_Tenis_Varias.jpg,basketball_court,football_field,football_field,football_field,basketball_court
5,15_Tenis_Varias.jpg,basketball_court,tennis_court,tennis_court,tennis_court,tennis_court
6,16_Multiples.JPG,baseball_field,tennis_court,football_field,tennis_court,basketball_court
7,17_Multiples.jpg,football_field,basketball_court,football_field,football_field,football_field
8,1_Baseball.tif,baseball_field,baseball_field,baseball_field,baseball_field,baseball_field
9,2_Park.jpg,basketball_court,basketball_court,football_field,football_field,golf_course


# Optimal

In [20]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import torch
from PIL import Image
import cv2
from torchvision import transforms

# Función para cargar modelos PyTorch
def cargar_modelo_pytorch(ruta):
    modelo = torch.load(ruta)
    modelo.eval()  # Pone el modelo en modo evaluación
    return modelo

# Definir el preprocesamiento para los modelos .pth
transform_pth = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Función para preprocesar imágenes para modelos .h5
def preprocesar_imagen_h5(img):
    img_redimensionada = redimensionar_imagen(img, (256, 256))
    img_normalizada = img_redimensionada / 255.0
    return np.expand_dims(img_normalizada, axis=0)

# Función para preprocesar imágenes para modelos .pth
def preprocesar_imagen_pth(img):
    img_tensor = transform_pth(img).unsqueeze(0)
    return img_tensor

# Función para redimensionar la imagen
def redimensionar_imagen(img, nuevo_tamanio):
    if img.dtype != np.uint8:
        img = (img * 255).astype(np.uint8)
    img_pil = Image.fromarray(img)
    img_redimensionada = img_pil.resize(nuevo_tamanio, Image.Resampling.LANCZOS)
    return np.array(img_redimensionada)

# Cargar la imagen desde la carpeta
def cargar_imagen_de_carpeta(ruta_carpeta, nombre_archivo):
    ruta_imagen = os.path.join(ruta_carpeta, nombre_archivo)
    img = cv2.imread(ruta_imagen)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Codificaciones para cada modelo
codificaciones = {
    'Optimal31_Custom': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
    'Optimal31_Mobile': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
    'Optimal31_Resnet': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
    'Optimal31_VG16': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
    'Optimal31_Xception': {0: 'baseball_field', 1: 'basketball_court', 2: 'football_field', 3: 'golf_course', 4: 'tennis_court'},
}

# Cargar modelos
modelos = {
    'Optimal31_Custom': load_model('C:/Users/nicol/Desktop/Modelos/Optimal31/Optimal31_Custom.h5'),
    'Optimal31_Mobile': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/Optimal31/Optimal31_mobileNET.pth'),
    'Optimal31_Resnet': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/Optimal31/Optimal31_ResNET18.pth'),
    'Optimal31_VG16': load_model('C:/Users/nicol/Desktop/Modelos/Optimal31/Optimal31_VGG16.h5'),
    'Optimal31_Xception': load_model('C:/Users/nicol/Desktop/Modelos/Optimal31/Optimal31_Xception.h5'),
}

ruta_carpeta = 'C:/Users/nicol/Desktop/Tests'
archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]

# Preparar DataFrame
df = pd.DataFrame(columns=['Imagen'] + list(modelos.keys()))
df['Imagen'] = archivos

# Realizar predicciones
for archivo in archivos:
    img = cargar_imagen_de_carpeta(ruta_carpeta, archivo)

    for nombre_modelo, modelo in modelos.items():
        if isinstance(modelo, tf.keras.Model):
            img_preprocesada = preprocesar_imagen_h5(img)
            pred = modelo.predict(img_preprocesada)
            pred_clase = np.argmax(pred, axis=1)[0]
        elif isinstance(modelo, torch.nn.Module):
            img_preprocesada = preprocesar_imagen_pth(img)
            with torch.no_grad():
                pred = modelo(img_preprocesada)
                pred_clase = pred.argmax(dim=1).item()
        
        # Convertir el índice predicho a la clase correspondiente usando la codificación
        clase_decodificada = codificaciones[nombre_modelo].get(pred_clase, 'Desconocido')
        df.loc[df['Imagen'] == archivo, nombre_modelo] = clase_decodificada

df


C:\Users\nicol\AppData\Local\Temp\ipykernel_11784\800672562.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo = torch.load(ruta)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 725ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/

,Imagen,Optimal31_Custom,Optimal31_Mobile,Optimal31_Resnet,Optimal31_VG16,Optimal31_Xception
0,10_Basket.jpg,baseball_field,basketball_court,basketball_court,basketball_court,basketball_court
1,11_Tenis.JPG,golf_course,basketball_court,basketball_court,basketball_court,basketball_court
2,12_Multiples_Basket_Tenis.JPG,golf_course,golf_course,basketball_court,basketball_court,basketball_court
3,13_Multiple_Basket_Tenis.jpg,golf_course,basketball_court,basketball_court,basketball_court,basketball_court
4,14_Tenis_Varias.jpg,golf_course,golf_course,basketball_court,basketball_court,basketball_court
5,15_Tenis_Varias.jpg,baseball_field,basketball_court,golf_course,basketball_court,basketball_court
6,16_Multiples.JPG,baseball_field,basketball_court,basketball_court,basketball_court,basketball_court
7,17_Multiples.jpg,baseball_field,basketball_court,basketball_court,golf_course,basketball_court
8,1_Baseball.tif,baseball_field,football_field,baseball_field,baseball_field,basketball_court
9,2_Park.jpg,baseball_field,football_field,football_field,football_field,football_field


# Mixto

In [22]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import torch
from PIL import Image
import cv2
from torchvision import transforms

# Función para cargar modelos PyTorch
def cargar_modelo_pytorch(ruta):
    modelo = torch.load(ruta)
    modelo.eval()  # Pone el modelo en modo evaluación
    return modelo

# Definir el preprocesamiento para los modelos .pth
transform_pth = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Función para preprocesar imágenes para modelos .h5
def preprocesar_imagen_h5(img):
    img_redimensionada = redimensionar_imagen(img, (256, 256))
    img_normalizada = img_redimensionada / 255.0
    return np.expand_dims(img_normalizada, axis=0)

# Función para preprocesar imágenes para modelos .pth
def preprocesar_imagen_pth(img):
    img_tensor = transform_pth(img).unsqueeze(0)
    return img_tensor

# Función para redimensionar la imagen
def redimensionar_imagen(img, nuevo_tamanio):
    if img.dtype != np.uint8:
        img = (img * 255).astype(np.uint8)
    img_pil = Image.fromarray(img)
    img_redimensionada = img_pil.resize(nuevo_tamanio, Image.Resampling.LANCZOS)
    return np.array(img_redimensionada)

# Cargar la imagen desde la carpeta
def cargar_imagen_de_carpeta(ruta_carpeta, nombre_archivo):
    ruta_imagen = os.path.join(ruta_carpeta, nombre_archivo)
    img = cv2.imread(ruta_imagen)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Codificaciones para cada modelo
codificaciones = {
    'Mixto_Custom': {0: 'baseball_diamond', 1: 'Basket', 2: 'Campo_Futbol', 3: 'Cancha_Multiple', 4: 'golf_course', 5: 'Parque', 6: 'Tenis'},
    'Mixto_Mobile': {0: 'baseball_diamond', 1: 'Basket', 2: 'Campo_Futbol', 3: 'Cancha_Multiple', 4: 'golf_course', 5: 'Parque', 6: 'Tenis'},
    'Mixto_Resnet': {0: 'baseball_diamond', 1: 'Basket', 2: 'Campo_Futbol', 3: 'Cancha_Multiple', 4: 'golf_course', 5: 'Parque', 6: 'Tenis'},
    'Mixto_VG16': {0: 'baseball_diamond', 1: 'Basket', 2: 'Campo_Futbol', 3: 'Cancha_Multiple', 4: 'golf_course', 5: 'Parque', 6: 'Tenis'},
    'Mixto_Xception': {0: 'baseball_diamond', 1: 'Basket', 2: 'Campo_Futbol', 3: 'Cancha_Multiple', 4: 'golf_course', 5: 'Parque', 6: 'Tenis'},
}

# Cargar modelos
modelos = {
    'Mixto_Custom': load_model('C:/Users/nicol/Desktop/Modelos/Mixto/Mixto_Custom.h5'),
    'Mixto_Mobile': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/Mixto/Mixto_mobileNET.pth'),
    'Mixto_Resnet': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/Mixto/Mixto_ResNET18.pth'),
    'Mixto_VG16': load_model('C:/Users/nicol/Desktop/Modelos/Mixto/Mixto_VGG16.h5'),
    'Mixto_Xception': load_model('C:/Users/nicol/Desktop/Modelos/Mixto/Mixto_Xception.h5'),
}

ruta_carpeta = 'C:/Users/nicol/Desktop/Tests'
archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]

# Preparar DataFrame
df = pd.DataFrame(columns=['Imagen'] + list(modelos.keys()))
df['Imagen'] = archivos

# Realizar predicciones
for archivo in archivos:
    img = cargar_imagen_de_carpeta(ruta_carpeta, archivo)

    for nombre_modelo, modelo in modelos.items():
        if isinstance(modelo, tf.keras.Model):
            img_preprocesada = preprocesar_imagen_h5(img)
            pred = modelo.predict(img_preprocesada)
            pred_clase = np.argmax(pred, axis=1)[0]
        elif isinstance(modelo, torch.nn.Module):
            img_preprocesada = preprocesar_imagen_pth(img)
            with torch.no_grad():
                pred = modelo(img_preprocesada)
                pred_clase = pred.argmax(dim=1).item()
        
        # Convertir el índice predicho a la clase correspondiente usando la codificación
        clase_decodificada = codificaciones[nombre_modelo].get(pred_clase, 'Desconocido')
        df.loc[df['Imagen'] == archivo, nombre_modelo] = clase_decodificada

df


C:\Users\nicol\AppData\Local\Temp\ipykernel_11784\190277264.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo = torch.load(ruta)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 689ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1

,Imagen,Mixto_Custom,Mixto_Mobile,Mixto_Resnet,Mixto_VG16,Mixto_Xception
0,10_Basket.jpg,Basket,Basket,Basket,Basket,Basket
1,11_Tenis.JPG,Basket,Tenis,Tenis,Tenis,Tenis
2,12_Multiples_Basket_Tenis.JPG,Basket,Basket,Basket,Basket,Basket
3,13_Multiple_Basket_Tenis.jpg,Campo_Futbol,Tenis,Tenis,Basket,Tenis
4,14_Tenis_Varias.jpg,Basket,Tenis,Tenis,Tenis,Tenis
5,15_Tenis_Varias.jpg,Basket,Tenis,Tenis,Tenis,Tenis
6,16_Multiples.JPG,Campo_Futbol,Tenis,Campo_Futbol,Tenis,Campo_Futbol
7,17_Multiples.jpg,Campo_Futbol,Campo_Futbol,Campo_Futbol,Cancha_Multiple,Campo_Futbol
8,1_Baseball.tif,baseball_diamond,baseball_diamond,baseball_diamond,baseball_diamond,baseball_diamond
9,2_Park.jpg,Basket,Parque,Parque,Parque,Parque


# WHU

In [24]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import torch
from PIL import Image
import cv2
from torchvision import transforms

# Función para cargar modelos PyTorch
def cargar_modelo_pytorch(ruta):
    modelo = torch.load(ruta)
    modelo.eval()  # Pone el modelo en modo evaluación
    return modelo

# Definir el preprocesamiento para los modelos .pth
transform_pth = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Función para preprocesar imágenes para modelos .h5
def preprocesar_imagen_h5(img):
    img_redimensionada = redimensionar_imagen(img, (256, 256))
    img_normalizada = img_redimensionada / 255.0
    return np.expand_dims(img_normalizada, axis=0)

# Función para preprocesar imágenes para modelos .pth
def preprocesar_imagen_pth(img):
    img_tensor = transform_pth(img).unsqueeze(0)
    return img_tensor

# Función para redimensionar la imagen
def redimensionar_imagen(img, nuevo_tamanio):
    if img.dtype != np.uint8:
        img = (img * 255).astype(np.uint8)
    img_pil = Image.fromarray(img)
    img_redimensionada = img_pil.resize(nuevo_tamanio, Image.Resampling.LANCZOS)
    return np.array(img_redimensionada)

# Cargar la imagen desde la carpeta
def cargar_imagen_de_carpeta(ruta_carpeta, nombre_archivo):
    ruta_imagen = os.path.join(ruta_carpeta, nombre_archivo)
    img = cv2.imread(ruta_imagen)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Codificaciones para cada modelo
codificaciones = {
    'WHU_Custom': {0: 'baseballdiamond', 1: 'footballField', 2: 'golfcourse', 3: 'Park', 4: 'Pond', 5: 'tenniscourt'},
    'WHU_Mobile': {0: 'baseballdiamond', 1: 'footballField', 2: 'golfcourse', 3: 'Park', 4: 'Pond', 5: 'tenniscourt'},
    'WHU_Resnet': {0: 'baseballdiamond', 1: 'footballField', 2: 'golfcourse', 3: 'Park', 4: 'Pond', 5: 'tenniscourt'},
    'WHU_VG16': {0: 'baseballdiamond', 1: 'footballField', 2: 'golfcourse', 3: 'Park', 4: 'Pond', 5: 'tenniscourt'},
    'WHU_Xception': {0: 'baseballdiamond', 1: 'footballField', 2: 'golfcourse', 3: 'Park', 4: 'Pond', 5: 'tenniscourt'},
}

# Cargar modelos
modelos = {
    'WHU_Custom': load_model('C:/Users/nicol/Desktop/Modelos/WHU/WHU_Custom.h5'),
    'WHU_Mobile': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/WHU/WHU_mobileNET.pth'),
    'WHU_Resnet': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/WHU/WHU_ResNET18.pth'),
    'WHU_VG16': load_model('C:/Users/nicol/Desktop/Modelos/WHU/WHU_VGG16.h5'),
    'WHU_Xception': load_model('C:/Users/nicol/Desktop/Modelos/WHU/WHU_Xception.h5'),
}

ruta_carpeta = 'C:/Users/nicol/Desktop/Tests'
archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]

# Preparar DataFrame
df = pd.DataFrame(columns=['Imagen'] + list(modelos.keys()))
df['Imagen'] = archivos

# Realizar predicciones
for archivo in archivos:
    img = cargar_imagen_de_carpeta(ruta_carpeta, archivo)

    for nombre_modelo, modelo in modelos.items():
        if isinstance(modelo, tf.keras.Model):
            img_preprocesada = preprocesar_imagen_h5(img)
            pred = modelo.predict(img_preprocesada)
            pred_clase = np.argmax(pred, axis=1)[0]
        elif isinstance(modelo, torch.nn.Module):
            img_preprocesada = preprocesar_imagen_pth(img)
            with torch.no_grad():
                pred = modelo(img_preprocesada)
                pred_clase = pred.argmax(dim=1).item()
        
        # Convertir el índice predicho a la clase correspondiente usando la codificación
        clase_decodificada = codificaciones[nombre_modelo].get(pred_clase, 'Desconocido')
        df.loc[df['Imagen'] == archivo, nombre_modelo] = clase_decodificada

df


C:\Users\nicol\AppData\Local\Temp\ipykernel_11784\239921132.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo = torch.load(ruta)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━

,Imagen,WHU_Custom,WHU_Mobile,WHU_Resnet,WHU_VG16,WHU_Xception
0,10_Basket.jpg,baseballdiamond,baseballdiamond,tenniscourt,tenniscourt,tenniscourt
1,11_Tenis.JPG,tenniscourt,tenniscourt,tenniscourt,tenniscourt,tenniscourt
2,12_Multiples_Basket_Tenis.JPG,tenniscourt,tenniscourt,tenniscourt,tenniscourt,tenniscourt
3,13_Multiple_Basket_Tenis.jpg,tenniscourt,Park,Park,tenniscourt,tenniscourt
4,14_Tenis_Varias.jpg,tenniscourt,golfcourse,baseballdiamond,tenniscourt,tenniscourt
5,15_Tenis_Varias.jpg,tenniscourt,tenniscourt,tenniscourt,tenniscourt,tenniscourt
6,16_Multiples.JPG,tenniscourt,tenniscourt,footballField,tenniscourt,tenniscourt
7,17_Multiples.jpg,tenniscourt,tenniscourt,tenniscourt,tenniscourt,tenniscourt
8,1_Baseball.tif,baseballdiamond,baseballdiamond,baseballdiamond,baseballdiamond,baseballdiamond
9,2_Park.jpg,golfcourse,Park,Park,Park,golfcourse


# MLRSNet

In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import torch
from PIL import Image
import cv2
from torchvision import transforms

# Función para cargar modelos PyTorch
def cargar_modelo_pytorch(ruta):
    modelo = torch.load(ruta)
    modelo.eval()  # Pone el modelo en modo evaluación
    return modelo

# Definir el preprocesamiento para los modelos .pth
transform_pth = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Función para preprocesar imágenes para modelos .h5
def preprocesar_imagen_h5(img):
    img_redimensionada = redimensionar_imagen(img, (256, 256))
    img_normalizada = img_redimensionada / 255.0
    return np.expand_dims(img_normalizada, axis=0)

# Función para preprocesar imágenes para modelos .pth
def preprocesar_imagen_pth(img):
    img_tensor = transform_pth(img).unsqueeze(0)
    return img_tensor

# Función para redimensionar la imagen
def redimensionar_imagen(img, nuevo_tamanio):
    if img.dtype != np.uint8:
        img = (img * 255).astype(np.uint8)
    img_pil = Image.fromarray(img)
    img_redimensionada = img_pil.resize(nuevo_tamanio, Image.Resampling.LANCZOS)
    return np.array(img_redimensionada)

# Cargar la imagen desde la carpeta
def cargar_imagen_de_carpeta(ruta_carpeta, nombre_archivo):
    ruta_imagen = os.path.join(ruta_carpeta, nombre_archivo)
    img = cv2.imread(ruta_imagen)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Codificaciones para cada modelo
codificaciones = {
    
    'MLRSNet_Custom': {0 :'baseball_diamond', 1: 'basketball_court', 2: 'golf_course', 3:  'ground_track_field', 4: 'park',5:'stadium',6: 'tennis_court'},
    'MLRSNet_Mobile': {0 :'baseball_diamond', 1: 'basketball_court', 2: 'golf_course', 3:  'ground_track_field', 4: 'park',5:'stadium',6: 'tennis_court'},
    'MLRSNet_Resnet': {0 :'baseball_diamond', 1: 'basketball_court', 2: 'golf_course', 3:  'ground_track_field', 4: 'park',5:'stadium',6: 'tennis_court'},
    'MLRSNet_VG16': {0 :'baseball_diamond', 1: 'basketball_court', 2: 'golf_course', 3:  'ground_track_field', 4: 'park',5:'stadium',6: 'tennis_court'},
    'MLRSNet_Xception': {0 :'baseball_diamond', 1: 'basketball_court', 2: 'golf_course', 3:  'ground_track_field', 4: 'park',5:'stadium',6: 'tennis_court'},
}

# Cargar modelos
modelos = {
    'MLRSNet_Custom': load_model('C:/Users/nicol/Desktop/Modelos/MLSRNet/MLRSNet_Custom.h5'),
    'MLRSNet_Mobile': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/MLSRNet/MLRSNet_mobileNET.pth'),
    'MLRSNet_Resnet': cargar_modelo_pytorch('C:/Users/nicol/Desktop/Modelos/MLSRNet/MLRSNet_ResNET18.pth'),
    'MLRSNet_VG16': load_model('C:/Users/nicol/Desktop/Modelos/MLSRNet/MLRSNet_VGG16.h5'),
    'MLRSNet_Xception': load_model('C:/Users/nicol/Desktop/Modelos/MLSRNet/MLRSNet_Xception.h5'),
}

ruta_carpeta = 'C:/Users/nicol/Desktop/Tests'
archivos = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]

# Preparar DataFrame
df = pd.DataFrame(columns=['Imagen'] + list(modelos.keys()))
df['Imagen'] = archivos

# Realizar predicciones
for archivo in archivos:
    img = cargar_imagen_de_carpeta(ruta_carpeta, archivo)

    for nombre_modelo, modelo in modelos.items():
        if isinstance(modelo, tf.keras.Model):
            img_preprocesada = preprocesar_imagen_h5(img)
            pred = modelo.predict(img_preprocesada)
            pred_clase = np.argmax(pred, axis=1)[0]
        elif isinstance(modelo, torch.nn.Module):
            img_preprocesada = preprocesar_imagen_pth(img)
            with torch.no_grad():
                pred = modelo(img_preprocesada)
                pred_clase = pred.argmax(dim=1).item()
        
        # Convertir el índice predicho a la clase correspondiente usando la codificación
        clase_decodificada = codificaciones[nombre_modelo].get(pred_clase, 'Desconocido')
        df.loc[df['Imagen'] == archivo, nombre_modelo] = clase_decodificada

df


C:\Users\nicol\AppData\Local\Temp\ipykernel_3732\815154368.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo = torch.load(ruta)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/st

,Imagen,MLRSNet_Custom,MLRSNet_Mobile,MLRSNet_Resnet,MLRSNet_VG16,MLRSNet_Xception
0,10_Basket.jpg,stadium,tennis_court,tennis_court,tennis_court,tennis_court
1,11_Tenis.JPG,stadium,tennis_court,tennis_court,tennis_court,tennis_court
2,12_Multiples_Basket_Tenis.JPG,stadium,tennis_court,tennis_court,tennis_court,tennis_court
3,13_Multiple_Basket_Tenis.jpg,tennis_court,tennis_court,tennis_court,tennis_court,tennis_court
4,14_Tenis_Varias.jpg,stadium,tennis_court,tennis_court,tennis_court,tennis_court
5,15_Tenis_Varias.jpg,stadium,tennis_court,tennis_court,tennis_court,tennis_court
6,16_Multiples.JPG,stadium,tennis_court,tennis_court,tennis_court,tennis_court
7,17_Multiples.jpg,stadium,tennis_court,tennis_court,tennis_court,tennis_court
8,1_Baseball.tif,golf_course,baseball_diamond,baseball_diamond,baseball_diamond,baseball_diamond
9,2_Park.jpg,stadium,park,park,park,park
